In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import freqopttest.util as util
import freqopttest.data as data
import freqopttest.kernel as kernel
import freqopttest.tst as tst
import collections
from collections import defaultdict
import freqopttest.glo as glo
import sys
import time
import os
import pandas as pd
import pickle
from itertools import combinations
import itertools
from multiprocessing import Pool, cpu_count
import json
import os
import pickle
from concurrent.futures import ProcessPoolExecutor
from itertools import combinations



In [2]:

# functions:

def generate_column_pairs(num_columns):
    """
    Generates pairs of columns for sliding window analysis.

    Args:
        num_columns (int): The total number of columns.

    Yields:
        tuple: A tuple containing the start index, end index, and shift for a pair of columns.
               The start and end indices define the column pair, and the shift indicates the distance
               by which one column is shifted with respect to the other.
    """
    for window in range(1, num_columns):
        for shift in range(1, num_columns - window + 1):
            for start_point, end_point in itertools.combinations(range(num_columns), 2):
                if end_point - start_point == window:
                    yield start_point, end_point, shift


def create_kgauss(sigma2, default_sigma2=None):
    """
    Create a KGauss instance with the given sigma2 value, or use the default_sigma2 value if provided.

    :param sigma2: float, the sigma2 value to use for creating the KGauss instance.
    :param default_sigma2: float, optional, the default sigma2 value to use if the provided sigma2 is invalid.
    :return: KGauss, the created KGauss instance.
    :raise ValueError: if both sigma2 and default_sigma2 are invalid.
    """
    if sigma2 > 0:
        return kernel.KGauss(sigma2)
    elif default_sigma2 is not None and default_sigma2 > 0:
        print("Using default sigma2 value:", default_sigma2)
        return kernel.KGauss(default_sigma2)
    else:
        raise ValueError("Both sigma2 and default_sigma2 are invalid. Please provide a positive value for either.")


def simulate_null_spectral(weights, n_simulate=1000, seed=275):
    """
    weights: chi-square weights (for the infinite weigted sum of chi squares)
    Return the values of MMD^2 (NOT n*MMD^2) simulated from the null distribution by
    the spectral method.
    """
    # draw at most block_size values at a time
    block_size = 400
    D = weights.shape[0]  # len(weights)
    mmds = np.zeros(n_simulate)
    from_ind = 0

    with util.NumpySeedContext(seed=seed):
        while from_ind < n_simulate:
            to_draw = min(block_size, n_simulate - from_ind)
            # draw chi^2 random variables.
            chi2 = np.random.randn(D, to_draw) ** 2
            # an array of length to_draw
            sim_mmds = 2.0 * weights.dot(chi2 - 1.0)
            # store
            end_ind = from_ind + to_draw
            mmds[from_ind:end_ind] = sim_mmds
            from_ind = end_ind
    return mmds


def chi_square_weights_H0(k, X):
    """
    Return a numpy array of the weights to be used as the weights in the
    weighted sum of chi-squares for the null distribution of MMD^2.
    - k: a Kernel
    - X: n x d number array of n data points
    """
    n = X.shape[0]
    # Gram matrix
    K = k.eval(X, X)
    # centring matrix. Not the most efficient way.
    H = np.eye(n) - np.ones((n, n)) / float(n)
    HKH = H.dot(K).dot(H)
    # https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.linalg.eigvals.html
    evals = np.linalg.eigvals(HKH)
    evals = np.real(evals)
    # sort in decreasing order
    evals = -np.sort(-evals)
    weights = evals / float(n) ** 2
    return weights


In [10]:
class MMDTester:
    def __init__(self, df):
        self.df = df

    def get_data(self, start_point, end_point, shift, window):
        min_length = min(self.df.iloc[:, start_point:end_point].shape[0],
                         self.df.iloc[:, end_point + shift:end_point + 2 * shift].shape[0],
                         self.df.iloc[:, start_point + window:end_point + window].shape[0])

        X = np.array(self.df.iloc[:min_length, start_point:end_point])
        Y = np.array(self.df.iloc[:min_length, end_point + shift:end_point + 2 * shift])
        Z = np.array(self.df.iloc[:min_length, start_point + window:end_point + window])

        return X, Y, Z

    def perform_tests(self, X, Y, Z):
        data_sample = data.TSTData(X, Y)
        test_data_one = data.TSTData(X, Z)
        test_data_two = data.TSTData(Y, Z)

        tr, te = data_sample.split_tr_te(tr_proportion=0.9, seed=100)

        med = util.meddistance(tr.stack_xy())
        widths = [(med * f) for f in 2.0 ** np.linspace(0, 4, 25)]
        try:

            list_kernels = [kernel.KGauss(w ** 2) for w in widths]
            print('using these', list_kernels)
        except AssertionError:
            print('setting sigma2 =1 as sigma2 > 0, must be > 0')
            list_kernels = [create_kgauss(w ** 2, default_sigma2=1) for w in widths]

        besti, powers = tst.LinearMMDTest.grid_search_kernel(tr, list_kernels, alpha=0.01)

        plt.plot(widths, powers, 'o-')
        plt.xlabel('Gaussian width')
        plt.ylabel('test power')
        plt.title('median distance = %.3g. Best width: %.3g' % (med, widths[besti]))
        plt.show()

        best_ker = list_kernels[besti]
        lin_mmd_test = tst.LinearMMDTest(best_ker, alpha=0.01)

        test_results_one = {
            'widths': widths,
            'med': med,
            'besti': besti,
            'powers': powers,
            'med_on_test_data': util.meddistance(test_data_one.stack_xy()),
            'test_result': lin_mmd_test.perform_test(test_data_one),
            'test_variance': lin_mmd_test.variance(X, Z, best_ker),
            'two_moments': lin_mmd_test.two_moments(X, Z, best_ker),
            'compute_unbiased_linear_estimator': lin_mmd_test.compute_stat(test_data_one)
        }

        test_results_two = {
            'test_result': lin_mmd_test.perform_test(test_data_two),
            'test_variance': lin_mmd_test.variance(Y, Z, best_ker),
            'med_on_test_data': util.meddistance(test_data_two.stack_xy()),
            'two_moments': lin_mmd_test.two_moments(Y, Z, best_ker),
            'compute_unbiased_linear_estimator': lin_mmd_test.compute_stat(test_data_two)
        }

        return test_results_one, test_results_two

    def perform_quad_mmd_tests(self, start_point, end_point, shift, window):

        X, Y, Z = self.get_data(start_point, end_point, shift, window)

        # Initialize a dictionary to store the results
        mmd_train_test_results = defaultdict(dict)

        try:
            tr_data = data.TSTData(X, Y)
            test_data_one = data.TSTData(X, Z)
            test_data_two = data.TSTData(Y, Z)

            # training dictionary results
            tr, te = tr_data.split_tr_te(tr_proportion=0.95, seed=10)  # is this necessary?!

            xtr, ytr = tr.xy()
            xytr = tr.stack_xy()
            sig2 = util.meddistance(xytr, subsample=1000)
            k = kernel.KGauss(sig2)
            mean, var = tst.QuadMMDTest.h1_mean_var(xtr, ytr, k, is_var_computed=True)
            Kx = k.eval(xtr, xtr)
            Ky = k.eval(ytr, ytr)
            Kxy = k.eval(xtr, ytr)
            mean_gram, var_gram = tst.QuadMMDTest.h1_mean_var_gram(Kx, Ky, Kxy, k, True)
            chi2_weights = chi_square_weights_H0(k, xytr)
            sim_mmds = simulate_null_spectral(chi2_weights, n_simulate=2000)
            # choose the best parameter and perform a test with permutations
            med = util.meddistance(tr.stack_xy(), 1000)
            list_gwidth = np.hstack(((med ** 2) * (2.0 ** np.linspace(-4, 4, 20))))
            list_gwidth.sort()
            list_kernels = [kernel.KGauss(gw2) for gw2 in list_gwidth]
            list_kernels_verbose = [kernel.KGauss(gw2).__str__() for gw2 in list_gwidth]

            # grid search to choose the best Gaussian width
            besti, powers = tst.QuadMMDTest.grid_search_kernel(tr, list_kernels, alpha=0.05)
            # perform test
            best_ker = list_kernels[besti]
            mmd_train_test_results[start_point]['perm_mmds1'] = tst.QuadMMDTest.permutation_list_mmd2(xtr, ytr, k,
                                                                                                      n_permute=2000)

            # Save the results in the mmd_train_test_results dictionary
            mmd_train_test_results[start_point]['perm_mmds1'] = tst.QuadMMDTest.permutation_list_mmd2(xtr, ytr, k,
                                                                                                      n_permute=2000)
            mmd_train_test_results[start_point]['chi2_weights'] = chi2_weights
            mmd_train_test_results[start_point]['sim_mmds'] = sim_mmds
            mmd_train_test_results[start_point]['sig2'] = sig2
            mmd_train_test_results[start_point]['Kxy'] = k.eval(xtr, ytr)
            mmd_train_test_results[start_point]['mean'] = mean
            mmd_train_test_results[start_point]['var'] = var
            mmd_train_test_results[start_point]['Kxx'] = k.eval(xtr, xtr)
            mmd_train_test_results[start_point]['Kyy'] = k.eval(ytr, ytr)
            mmd_train_test_results[start_point]['mean_gram'] = mean_gram
            mmd_train_test_results[start_point]['var_gram'] = var_gram
            mmd_train_test_results[start_point]['med'] = util.meddistance(tr.stack_xy(), 1000)
            mmd_train_test_results[start_point]['list_gwidth'] = list_gwidth.sort()
            mmd_train_test_results[start_point]['list_kernels'] = list_kernels_verbose
            mmd_train_test_results[start_point]['besti'] = besti
            mmd_train_test_results[start_point]['powers'] = powers
            mmd_train_test_results[start_point]['best_ker'] = best_ker.__str__()

            alpha = 0.05
            mmd_test = tst.QuadMMDTest(best_ker, n_permute=2000, alpha=alpha)
            mmd_train_test_results[start_point]['XZ_test'] = mmd_test.perform_test(test_data_one)
            mmd_train_test_results[start_point]['YZ_test'] = mmd_test.perform_test(test_data_two)
        except ValueError:
            pass

        return mmd_train_test_results

    def analyze(self, start_point, end_point, shift, window):
        X, Y, Z = self.get_data(start_point, end_point, shift, window)
        return self.perform_tests(X, Y, Z)


def analyze_column(mmd_tester, unpickled_df, start_point, end_point, shift, window):
    try:
        test_results_one, test_results_two = mmd_tester.analyze(start_point, end_point, shift, window)
        col1, col2 = unpickled_df.columns[start_point], unpickled_df.columns[end_point + shift]
        result_key = f"{col1} vs {col2}, window={window}, shift={shift}"
        return {result_key: (test_results_one, test_results_two)}
    except ValueError:
        pass


class QuadMMDAnalysis(MMDTester):
    def __init__(self, df, symbol, LinearMMDOutputFiles, bar_choice, variable):
        """
        Initialize QuadMMDAnalysis with the input data frame and other parameters.

        :param df: Input data frame containing the data to be analyzed.
        :param symbol: A string representing the symbol of the financial instrument.
        :param output_directory: A string representing the path to the output directory.
        :param bar_choice: A string representing the choice of bars (e.g., "BarChoice").
        :param variable: A string representing the variable (e.g., "Variable").
               """

        self.df = df
        self.symbol = symbol
        self.LinearMMDOutputFiles = LinearMMDOutputFiles
        self.bar_choice = bar_choice
        self.variable = variable

    def run_quad_mmd_analysis(self, start_point, end_point, shift, window):
        """
        Run the perform_quad_mmd_tests method with the parameters set as class attributes.

        :return: A dictionary containing the results of the quad MMD analysis.
        """
        # Run perform_quad_mmd_tests with the parameters set as class attributes
        mmd_train_test_results = self.perform_quad_mmd_tests(start_point, end_point, shift, window)

        # Print the results
        print(mmd_train_test_results)

    def run_single_analysis(self, start_point, end_point, shift, window):
        """
        Run the perform_quad_mmd_tests method for a single combination of start_point, end_point, shift, and window.
        run_single_analysis that takes the start_point, end_point, shift, and window as arguments and returns a tuple containing the combination and its corresponding result.

        :param start_point: The starting index of the column pair.
        :param end_point: The ending index of the column pair.
        :param shift: The shift to be applied during the analysis.
        :param window: The window size to be used during the analysis.
        :return: A tuple containing the combination of start_point, end_point, shift, window, and the result.
        """
        self.start_point = start_point
        self.end_point = end_point
        self.shift = shift
        self.window = window
        result = self.run_quad_mmd_analysis(self.start_point, self.end_point, self.shift, self.window)

        return (start_point, end_point, shift, window, result)

    def run_multiple_quad_mmd_analyses(self, windows, shifts, n_jobs=-1):
        """
        Run the perform_quad_mmd_tests method for multiple combinations of shifts and windows with parallelization.

        :param windows: A list of integers representing the window sizes to be used during the analysis.
        :param shifts: A list of integers representing the shifts to be applied during the analysis.
        :param n_jobs: The number of parallel processes to be used during the analysis.
        :return: A dictionary containing the results of the quad MMD analysis for all combinations.
        """
        # Generate all possible combinations of column pairs, window sizes, and shifts
        column_pairs = list(combinations(self.df.columns, 2))

        # Create a list of arguments for the analyze_column function
        args_list = []
        for start_point, end_point in column_pairs:
            for window in windows:
                for shift in shifts:
                    args_list.append((start_point, end_point, shift, window))

        # Run quad MMD analysis for all combinations and store the results in a dictionary
        results_dict = {}
        if n_jobs == -1:
            n_jobs = os.cpu_count() or 1

        with ProcessPoolExecutor(max_workers=n_jobs) as executor:
            for args, result in executor.map(lambda args: self.run_single_analysis(*args), args_list):
                results_dict[args] = result

        return results_dict

        # Modify the run_quad_mmd_analysis method to use smaller ranges

    def faster_run_quad_mmd_analysis(self):
        num_columns = self.df.shape[1]

        # Generate all possible combinations of column pairs, window sizes, and shifts with smaller ranges
        windows = range(5, 101, 10)
        shifts = range(1, 6)
        args_list = []
        result_dict = {}
        for start_point, end_point in list(combinations(range(len(self.df.columns)), 2))[:10]:  # Limit to 10 pairs
            for window in windows:
                for shift in shifts:
                    args_list.append((start_point, end_point, shift, window, result_dict))
#         print("Args list:", args_list)  # Add this print statement

        # Use multiprocessing to analyze the data in parallel
        with Pool(processes=cpu_count()) as pool:
#             results = pool.starmap(self.analyze_mmd_column, [(start_point, end_point, shift, window, self, {}) for
#                                                              start_point, end_point, shift, window in args_list])
            results = pool.starmap(self.analyze_mmd_column, args_list)  # Removed the extra argument in the tuple

        # Combine the results into a single dictionary
        result_dict = {}
        for result in results: # change this bit here !!! <---
            try:
                key, value = result
                if value:
                    result_dict[key] = value
            except TypeError:
                pass

        # Save the results to a pickle file
        output_file = os.path.join(self.LinearMMDOutputFiles,
                                   f"{self.symbol}_{self.bar_choice}_processedQUADMMDresults_{self.variable}.pickle")

        result_list = []
        for key, value in result_dict.items():
            col_pair = f"{self.df.columns[key[0]]} vs {self.df.columns[key[1] + key[2]]}"
            window, shift = key[3], key[2]
            result_list.append({
                "Column Pair": col_pair,
                "Window": window,
                "Shift": shift,
                "Test Results 1": value["Test Results 1"],
                "Test Results 2": value["Test Results 2"],
            })

        with open(output_file, "wb") as f:
            pickle.dump(result_list, f)

        print("Results saved to:", output_file)

    def analyze_mmd_column(self, start_point, end_point, shift, window):
        result_dict = {}
        try:
            result = self.run_single_analysis(start_point, end_point, shift, window)
            result_dict[(start_point, end_point, shift, window)] = result
        except ValueError:
            pass
        return result_dict

    def analyze_mmd_column(self, start_point, end_point, shift, window, result_dict):
        try:
            test_results_one, test_results_two = self.run_single_analysis(start_point, end_point, shift, window)
            col1, col2 = self.df.columns[start_point], self.df.columns[end_point + shift]
            result_key = f"{col1} vs {col2}, window={window}, shift={shift}"
            result_dict[(start_point, end_point, shift, window)] = {'Test Results 1': test_results_one,
                                                                    'Test Results 2': test_results_two}
        except ValueError:
            pass

    def analyze_column(self, start_point, end_point, shift, window, mmd_tester):
        try:
            test_results_one, test_results_two = mmd_tester.analyze(start_point, end_point, shift, window)
            col1, col2 = mmd_tester.df.columns[start_point], mmd_tester.df.columns[end_point + shift]
            result_key = f"{col1} vs {col2}, window={window}, shift={shift}"
            return {result_key: {'Test Results 1': test_results_one, 'Test Results 2': test_results_two}}
        except ValueError:
            pass

    def save_results_to_pickle(self, results, file_name):
        output_file = os.path.join(self.output_directory, file_name)

        with open(output_file, "wb") as f:
            pickle.dump(results, f)

        print("Results saved to:", output_file)


def analyze_mmd_column(self, start_point, end_point, shift, window):
    try:
        result = self.run_single_analysis(start_point, end_point, shift, window)
        col1, col2 = self.df.columns[start_point], self.df.columns[end_point + shift]
        result_key = f"{col1} vs {col2}, window={window}, shift={shift}"
        return ((start_point, end_point, shift, window), {'Test Results 1': result[0], 'Test Results 2': result[1]})
    except ValueError:
        pass

def analyze_column(unpickled_df, start_point, end_point, shift, window, mmd_tester):
    try:
        test_results_one, test_results_two = mmd_tester.analyze(start_point, end_point, shift, window)
        col1, col2 = unpickled_df.columns[start_point], unpickled_df.columns[end_point + shift]
        result_key = f"{col1} vs {col2}, window={window}, shift={shift}"
        return {result_key: {'Test Results 1': test_results_one, 'Test Results 2': test_results_two}}
    except ValueError:
        pass



In [4]:
# if __name__ == '__main__':
# need to load a dataframe here
symbol = 'XM1'
LinearMMDInputFiles = '/media/ak/T7/August11th2022Experiments/LinearMMDInputFiles/'
linear_mmd_output_files = '/media/ak/T7/August11th2022Experiments/QuadMMDOutputFiles'
bar_choice = 'tick'

outputDir = '/media/ak/T7/August11th2022Experiments/LinearMMDOutputFiles'
variables = ['n_F', 'list_H', 'list_H_intercept', 'tau', 'alpha', 'mfSpect']
file = os.path.join(LinearMMDInputFiles,
                    [f for f in os.listdir(LinearMMDInputFiles) if (str(symbol) and str(bar_choice)) in f][0])
data_dict = pd.read_pickle(file)

# Create a list of DataFrames with names from the dictionary keys
dataframes = [pd.DataFrame(data_dict[key]).assign(name=key) for key in data_dict]

n_f = data_dict['n_F']
list_H = data_dict['list_H']
tau_df = pd.DataFrame.from_dict(data_dict['tau'])
alpha_df = pd.DataFrame.from_dict(data_dict['alpha'])

# # # this is the processing code!
unpickled_dataframe = tau_df
variable = 'tau'  # bar_choice variable (based on above) - this is the mfdfa variable
output_name = "_".join((str(symbol), str(bar_choice), 'processedQUADMMDresults', str(variable)))
num_columns = unpickled_dataframe.shape[1]

In [5]:
quad_mmd_analysis = QuadMMDAnalysis(unpickled_dataframe, symbol, linear_mmd_output_files, bar_choice, variable)

In [6]:
start_point = 0
end_point = 1
shift = 1
window = 10
quad_mmd_analysis.run_quad_mmd_analysis(start_point, end_point, shift,window)

(1/20) KGauss(w2=0.079): mmd2: 0.0586, var: 0.000284, power obj: 1.63558, took: 0.00028395652770996094
(2/20) KGauss(w2=0.106): mmd2: 0.0491, var: 0.0001, power obj: 1.47973, took: 0.0004229545593261719
(3/20) KGauss(w2=0.142): mmd2: 0.0406, var: 1.89e-05, power obj: 1.27344, took: 0.0003027915954589844
(4/20) KGauss(w2=0.190): mmd2: 0.0335, var: 1.34e-05, power obj: 1.05255, took: 0.0002930164337158203
(5/20) KGauss(w2=0.255): mmd2: 0.0277, var: 9.33e-06, power obj: 0.871355, took: 0.00032591819763183594
(6/20) KGauss(w2=0.341): mmd2: 0.0231, var: 6.33e-06, power obj: 0.727597, took: 0.0002932548522949219
(7/20) KGauss(w2=0.456): mmd2: 0.0196, var: 4.2e-06, power obj: 0.617235, took: 0.00027871131896972656
(8/20) KGauss(w2=0.611): mmd2: 0.017, var: 2.74e-06, power obj: 0.535398, took: 0.00029087066650390625
(9/20) KGauss(w2=0.818): mmd2: 0.0151, var: 1.75e-06, power obj: 0.476286, took: 0.0002884864807128906
(10/20) KGauss(w2=1.096): mmd2: 0.0137, var: 1.11e-06, power obj: 0.432983, t

In [7]:
start_point = 0
end_point = 1
shift = 1
window = 10
quad_mmd_analysis.run_single_analysis(start_point = start_point, end_point= end_point, shift = shift, window =window)

(1/20) KGauss(w2=0.079): mmd2: 0.0586, var: 0.000284, power obj: 1.63558, took: 0.00030517578125
(2/20) KGauss(w2=0.106): mmd2: 0.0491, var: 0.0001, power obj: 1.47973, took: 0.00045371055603027344
(3/20) KGauss(w2=0.142): mmd2: 0.0406, var: 1.89e-05, power obj: 1.27344, took: 0.00034117698669433594
(4/20) KGauss(w2=0.190): mmd2: 0.0335, var: 1.34e-05, power obj: 1.05255, took: 0.00033402442932128906
(5/20) KGauss(w2=0.255): mmd2: 0.0277, var: 9.33e-06, power obj: 0.871355, took: 0.00034046173095703125
(6/20) KGauss(w2=0.341): mmd2: 0.0231, var: 6.33e-06, power obj: 0.727597, took: 0.0003364086151123047
(7/20) KGauss(w2=0.456): mmd2: 0.0196, var: 4.2e-06, power obj: 0.617235, took: 0.00048041343688964844
(8/20) KGauss(w2=0.611): mmd2: 0.017, var: 2.74e-06, power obj: 0.535398, took: 0.0004305839538574219
(9/20) KGauss(w2=0.818): mmd2: 0.0151, var: 1.75e-06, power obj: 0.476286, took: 0.0003521442413330078
(10/20) KGauss(w2=1.096): mmd2: 0.0137, var: 1.11e-06, power obj: 0.432983, took:

(0, 1, 1, 10, None)

In [8]:
# windows = [2, 4, 5]
# shifts = [1,2, 3]
# quad_mmd_analysis.run_multiple_quad_mmd_analyses(windows, shifts)


In [9]:

if __name__ == '__main__':
    # need to load a dataframe here
    symbol = 'XM1'
    LinearMMDInputFiles = '/media/ak/T7/August11th2022Experiments/LinearMMDInputFiles/'
    linear_mmd_output_files = '/media/ak/T7/August11th2022Experiments/QuadMMDOutputFiles'
    bar_choice = 'tick'

    outputDir = '/media/ak/T7/August11th2022Experiments/LinearMMDOutputFiles'
    variables = ['n_F', 'list_H', 'list_H_intercept', 'tau', 'alpha', 'mfSpect']
    file = os.path.join(LinearMMDInputFiles,
                        [f for f in os.listdir(LinearMMDInputFiles) if (str(symbol) and str(bar_choice)) in f][0])
    data_dict = pd.read_pickle(file)

    # Create a list of DataFrames with names from the dictionary keys
    dataframes = [pd.DataFrame(data_dict[key]).assign(name=key) for key in data_dict]

    n_f = data_dict['n_F']
    list_H = data_dict['list_H']
    tau_df = pd.DataFrame.from_dict(data_dict['tau'])
    alpha_df = pd.DataFrame.from_dict(data_dict['alpha'])

    # # # this is the processing code!
    unpickled_dataframe = tau_df
    variable = 'tau'  # bar_choice variable (based on above) - this is the mfdfa variable
    output_name = "_".join((str(symbol), str(bar_choice), 'processedQUADMMDresults', str(variable)))
    num_columns = unpickled_dataframe.shape[1]
    quad_mmd_analysis = QuadMMDAnalysis(unpickled_dataframe, symbol, linear_mmd_output_files, bar_choice, variable)
    # start_point = 0
    # end_point = 1
    # shift = 1
    # window = 10
    # quad_mmd_analysis.run_quad_mmd_analysis(start_point, end_point, shift, window)
    windows = [2, 4, 5]
    shifts = [1, 2, 3]
    # quad_mmd_analysis.run_multiple_quad_mmd_analyses(windows, shifts)
    quad_mmd_analysis.faster_run_quad_mmd_analysis()


defaultdict(<class 'dict'>, {})defaultdict(<class 'dict'>, {})defaultdict(<class 'dict'>, {})defaultdict(<class 'dict'>, {})defaultdict(<class 'dict'>, {})


defaultdict(<class 'dict'>, {})defaultdict(<class 'dict'>, {})defaultdict(<class 'dict'>, {})
defaultdict(<class 'dict'>, {})


defaultdict(<class 'dict'>, {})
defaultdict(<class 'dict'>, {})defaultdict(<class 'dict'>, {})
defaultdict(<class 'dict'>, {})defaultdict(<class 'dict'>, {})

defaultdict(<class 'dict'>, {})defaultdict(<class 'dict'>, {})

defaultdict(<class 'dict'>, {})defaultdict(<class 'dict'>, {})
defaultdict(<class 'dict'>, {})

defaultdict(<class 'dict'>, {})


defaultdict(<class 'dict'>, {})defaultdict(<class 'dict'>, {})
defaultdict(<class 'dict'>, {})
defaultdict(<class 'dict'>, {})defaultdict(<class 'dict'>, {})

(1/20) KGauss(w2=0.079): mmd2: 0.0586, var: 0.000284, power obj: 1.63558, took: 0.00058746337890625defaultdict(<class 'dict'>, {})

(1/20) KGauss(w2=0.079): mmd2: 0.0586, var: 0.000284, power obj: 1.635

(6/20) KGauss(w2=0.341): mmd2: 0.0231, var: 6.33e-06, power obj: 0.727597, took: 0.0004901885986328125

(3/20) KGauss(w2=0.436): mmd2: 0.0352, var: 9.78e-06, power obj: 1.10761, took: 0.0029892921447753906(10/20) KGauss(w2=1.096): mmd2: 0.0137, var: 1.11e-06, power obj: 0.432983, took: 0.00045680999755859375(4/20) KGauss(w2=0.367): mmd2: 0.0022, var: 3.99e-08, power obj: 0.0694106, took: 0.006087064743041992(6/20) KGauss(w2=0.341): mmd2: 0.0231, var: 6.33e-06, power obj: 0.727597, took: 0.0005643367767333984


(7/20) KGauss(w2=0.456): mmd2: 0.0196, var: 4.2e-06, power obj: 0.617235, took: 0.0005636215209960938(6/20) KGauss(w2=0.658): mmd2: 0.00122, var: 1.39e-08, power obj: 0.0385532, took: 0.0009489059448242188


(6/20) KGauss(w2=0.658): mmd2: 0.00122, var: 1.39e-08, power obj: 0.0385532, took: 0.013376235961914062
(7/20) KGauss(w2=0.881): mmd2: 0.000893, var: 8.22e-09, power obj: 0.0282505, took: 0.000997781753540039(8/20) KGauss(w2=1.876): mmd2: 0.0173, var: 1.87e-06, power obj: 0.5

(9/20) KGauss(w2=1.579): mmd2: 0.000464, var: 2.86e-09, power obj: 0.0146853, took: 0.0009675025939941406(8/20) KGauss(w2=1.180): mmd2: 0.000647, var: 4.85e-09, power obj: 0.0204642, took: 0.0022602081298828125(11/20) KGauss(w2=1.467): mmd2: 0.0126, var: 7.01e-07, power obj: 0.398147, took: 0.000476837158203125



(7/20) KGauss(w2=1.401): mmd2: 0.0205, var: 2.79e-06, power obj: 0.646227, took: 0.0009725093841552734(13/20) KGauss(w2=2.630): mmd2: 0.0104, var: 2.84e-07, power obj: 0.328169, took: 0.0004792213439941406
(9/20) KGauss(w2=0.818): mmd2: 0.0151, var: 1.75e-06, power obj: 0.476286, took: 0.0004875659942626953
(11/20) KGauss(w2=2.831): mmd2: 0.000238, var: 9.82e-10, power obj: 0.00753265, took: 0.0008816719055175781(15/20) KGauss(w2=4.714): mmd2: 0.00739, var: 1.14e-07, power obj: 0.233559, took: 0.0005240440368652344


(8/20) KGauss(w2=1.876): mmd2: 0.0173, var: 1.87e-06, power obj: 0.546033, took: 0.0009770393371582031(7/20) KGauss(w2=0.881): mmd2: 0.000893, var: 8.22e-09, pow

(12/20) KGauss(w2=6.027): mmd2: 0.00758, var: 3.04e-07, power obj: 0.239566, took: 0.0012135505676269531
(12/20) KGauss(w2=6.027): mmd2: 0.00758, var: 3.04e-07, power obj: 0.239566, took: 0.0008418560028076172
(17/20) KGauss(w2=8.451): mmd2: 0.0042, var: 4.06e-08, power obj: 0.13291, took: 0.0004932880401611328


(19/20) KGauss(w2=15.150): mmd2: 0.00197, var: 1.32e-08, power obj: 0.0622463, took: 0.0004947185516357422(13/20) KGauss(w2=2.630): mmd2: 0.0104, var: 2.84e-07, power obj: 0.328169, took: 0.00051116943359375(16/20) KGauss(w2=6.312): mmd2: 0.00573, var: 6.93e-08, power obj: 0.181212, took: 0.0005328655242919922

(13/20) KGauss(w2=8.070): mmd2: 0.0059, var: 1.85e-07, power obj: 0.186532, took: 0.0008027553558349609(19/20) KGauss(w2=46.492): mmd2: 0.00095, var: 7.51e-09, power obj: 0.0300351, took: 0.008778095245361328

(20/20) KGauss(w2=20.284): mmd2: 0.00128, var: 7.69e-09, power obj: 0.0405181, took: 0.0005290508270263672
(14/20) KGauss(w2=6.796): mmd2: 8.88e-05, var: 1.93e-10

(9/20) KGauss(w2=1.579): mmd2: 0.000464, var: 2.86e-09, power obj: 0.0146853, took: 0.0010843276977539062(19/20) KGauss(w2=46.492): mmd2: 0.00095, var: 7.51e-09, power obj: 0.0300351, took: 0.0022537708282470703


(20/20) KGauss(w2=39.152): mmd2: 2.73e-05, var: 6.82e-12, power obj: 0.000863402, took: 0.009301900863647461(20/20) KGauss(w2=20.284): mmd2: 0.00128, var: 7.69e-09, power obj: 0.0405181, took: 0.0005981922149658203(20/20) KGauss(w2=62.248): mmd2: 0.000686, var: 4.31e-09, power obj: 0.021703, took: 0.0010073184967041016(10/20) KGauss(w2=2.115): mmd2: 0.000332, var: 1.68e-09, power obj: 0.0104999, took: 0.0012664794921875
(20/20) KGauss(w2=62.248): mmd2: 0.000686, var: 4.31e-09, power obj: 0.021703, took: 0.000804901123046875



(11/20) KGauss(w2=2.831): mmd2: 0.000238, var: 9.82e-10, power obj: 0.00753265, took: 0.0009982585906982422
(12/20) KGauss(w2=3.791): mmd2: 0.000172, var: 5.72e-10, power obj: 0.00542724, took: 0.0013208389282226562
(13/20) KGauss(w2=5.076): mmd2: 0.000

...

        0.00109971, -0.01163684])}}})(6/20) KGauss(w2=1.327): mmd2: 0.0192, var: 1.82e-06, power obj: 0.606842, took: 0.0007843971252441406
(7/20) KGauss(w2=2.225): mmd2: 0.0137, var: 1.04e-06, power obj: 0.434089, took: 0.005213022232055664

(7/20) KGauss(w2=1.776): mmd2: 0.0156, var: 1.16e-06, power obj: 0.492916, took: 0.0005841255187988281(8/20) KGauss(w2=2.979): mmd2: 0.0114, var: 6.79e-07, power obj: 0.360162, took: 0.01637411117553711
defaultdict(<class 'dict'>, {})
(8/20) KGauss(w2=2.378): mmd2: 0.0126, var: 7.3e-07, power obj: 0.398565, took: 0.0023241043090820312

(9/20) KGauss(w2=3.988): mmd2: 0.00936, var: 4.35e-07, power obj: 0.295815, took: 0.009381771087646484(9/20) KGauss(w2=3.184): mmd2: 0.0101, var: 4.55e-07, power obj: 0.320531, took: 0.0012688636779785156defaultdict(<class 'dict'>, {})

(10/20) KGauss(w2=4.263): mmd2: 0.00809, var: 2.8e-07, power obj: 0.255935, took: 0.0015711784362792969
(11/20) KGauss(w2=5.708): mmd2: 0.0064, var: 1.71e-07, power obj: 0.202349, to

        0.00034934, -0.01253783])}}})

defaultdict(<class 'dict'>, {})
(18/20) KGauss(w2=55.144): mmd2: 0.000754, var: 4.53e-09, power obj: 0.0238336, took: 0.019541025161743164defaultdict(<class 'dict'>, {})

defaultdict(<class 'dict'>, {})
defaultdict(<class 'dict'>, {})(19/20) KGauss(w2=73.832): mmd2: 0.000532, var: 2.63e-09, power obj: 0.0168224, took: 0.006412982940673828
defaultdict(<class 'dict'>, {})
defaultdict(<class 'dict'>, {})

(20/20) KGauss(w2=98.854): mmd2: 0.000376, var: 1.53e-09, power obj: 0.0118814, took: 0.010847330093383789
defaultdict(<class 'dict'>, {0: {'perm_mmds1': array([-0.00199735, -0.0025237 ,  0.01416822, ..., -0.01160191,
        0.0165918 , -0.00784864]), 'chi2_weights': array([ 2.27283639e-03,  1.09927169e-03,  7.23183775e-04,  3.76960514e-04,
        1.68519063e-04,  7.72016717e-05,  5.94201748e-05,  4.33190359e-05,
        2.80265289e-05,  2.09818172e-05,  1.04594122e-05,  6.56919615e-06,
        3.28917902e-06,  1.83820306e-06,  1.09643860e-06,  6.

        0.00088559, -0.01138117])}}})
defaultdict(<class 'dict'>, {})
defaultdict(<class 'dict'>, {})
defaultdict(<class 'dict'>, {0: {'perm_mmds1': array([-0.00206608, -0.00234333,  0.01210269, ..., -0.01189345,
        0.01629406, -0.00707035]), 'chi2_weights': array([ 2.24397848e-03,  1.07739295e-03,  7.49475152e-04,  4.09254915e-04,
        1.96300385e-04,  9.28271761e-05,  7.53938231e-05,  3.68864167e-05,
        3.08599285e-05,  2.24673735e-05,  1.42797171e-05,  1.08913449e-05,
        4.61236581e-06,  3.38838950e-06,  1.57168115e-06,  9.37737497e-07,
        5.83156787e-07,  4.60611135e-07,  2.34676185e-07,  1.31269076e-07,
        1.24355261e-07,  5.90180020e-08,  4.84766697e-08,  3.49907288e-08,
        2.58945316e-08,  1.74182320e-08,  7.90937952e-09,  6.43458242e-09,
        2.81528315e-09,  1.63295403e-09,  7.85345353e-10,  3.74017287e-10,
        2.31706803e-10,  8.76226048e-11,  7.68561360e-11,  2.74297878e-11,
        2.54681129e-11,  7.42481622e-12,  6.93388444e-12,  2.

        0.00021341, -0.01203881])}}})
defaultdict(<class 'dict'>, {})
defaultdict(<class 'dict'>, {0: {'perm_mmds1': array([-0.00206608, -0.00234333,  0.01210269, ..., -0.01189345,
        0.01629406, -0.00707035]), 'chi2_weights': array([ 2.24397848e-03,  1.07739295e-03,  7.49475152e-04,  4.09254915e-04,
        1.96300385e-04,  9.28271761e-05,  7.53938231e-05,  3.68864167e-05,
        3.08599285e-05,  2.24673735e-05,  1.42797171e-05,  1.08913449e-05,
        4.61236581e-06,  3.38838950e-06,  1.57168115e-06,  9.37737497e-07,
        5.83156787e-07,  4.60611135e-07,  2.34676185e-07,  1.31269076e-07,
        1.24355261e-07,  5.90180020e-08,  4.84766697e-08,  3.49907288e-08,
        2.58945316e-08,  1.74182320e-08,  7.90937952e-09,  6.43458242e-09,
        2.81528315e-09,  1.63295403e-09,  7.85345353e-10,  3.74017287e-10,
        2.31706803e-10,  8.76226048e-11,  7.68561360e-11,  2.74297878e-11,
        2.54681129e-11,  7.42481622e-12,  6.93388444e-12,  2.04227872e-12,
        1.55188915

TypeError: cannot unpack non-iterable NoneType object